In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix
import warnings
warnings.filterwarnings('ignore')

In [24]:
df = pd.read_csv ('/content/drive/MyDrive/Recommendation/dfn3share.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head()

,user,service,count,subcat,gender
0,3646,92,127,15,1
1,3646,94,18,15,1
2,3646,93,19,15,1
3,3646,95,2,17,1
4,3646,114,1,15,1


In [25]:
min_counting=1
df_top_services = df[df['count']>=min_counting]
df_counting_with_top_services = df[df['service'].isin(list(df_top_services.index))]
df_counting_with_top_services.head()

,user,service,count,subcat,gender
0,3646,92,127,15,1
1,3646,94,18,15,1
2,3646,93,19,15,1
3,3646,95,2,17,1
4,3646,114,1,15,1


In [26]:
min_service_rated = 1
df_users = df_counting_with_top_services[['service','user']].groupby(['user']).agg(['count']).sort_values(('service','count'),ascending=False)
df_top_rating_users = df_users[df_users[('service', 'count')]>=min_service_rated]
top_rating_users = list(df_top_rating_users.index)
df_final=df_counting_with_top_services[df_counting_with_top_services['user'].isin(top_rating_users)]

# اگر این جا مقادیر نان با صفر پر شوند نتیجه متفاوت خواهد شد لطفا این موضوع را هم چک کنین

In [27]:
#df_user_item_matrix = df_final.pivot(index='user',columns='service',values='count').fillna(0)
df_user_item_matrix = df_final.pivot(index='user',columns='service',values='count')

In [28]:
df_user_item_matrix.head()

service,0,1,2,3,4,5,6,7,8,9,...,555,560,562,568,580,581,583,585,586,588
user,,,,,,,,,,,,,,,,,,,,,
0,3.0,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9.0,1.0,NaN,1.0,2.0,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
for selected_service in df_user_item_matrix.columns[:5]:
    result = df_user_item_matrix.corrwith(df_user_item_matrix[selected_service]).sort_values(ascending=False)
    recom_services = result[result.index != selected_service].head(10)

    # Append results to dataframe
    temp_df = pd.DataFrame({'service': [selected_service] * len(recom_services), 'recomservice': recom_services.index, 'corr': recom_services.values})
    result_df = result_df.append(temp_df, ignore_index=True)

# Save results to CSV file
result_df.to_csv('/content/drive/MyDrive/Final_Recommender/Matrix/recommendations.csv', index=False)